In [1]:
import argparse
import sys
import os
import shutil
import tempfile
import mlflow
from mlflow import pyfunc, tensorflow
import pandas as pd
import tensorflow as tf

In [6]:
batch_size = 100
train_steps = 1000

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
mlflow.set_experiment('experiment-tf2')

INFO: 'experiment-tf2' does not exist. Creating a new experiment


In [4]:
def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features = dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [7]:
with mlflow.start_run():
    tensorflow.autolog()
    #mlflow.set_tracking_uri("http://mlflow:5000")
    # Fetch the data
    (train_x, train_y), (test_x, test_y) = load_data()
    
    # Feature columns describe how to use the input.
    my_feature_columns = []
    for key in list(train_x.keys()):
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    # Two hidden layers of 10 nodes each.
    hidden_units = [10, 10]

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=hidden_units,
        # The model must choose between 3 classes.
        n_classes=3)

    # Train the Model.
    classifier.train(
        input_fn=lambda: train_input_fn(train_x, train_y,
                                        batch_size),
        steps=train_steps)

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda: eval_input_fn(test_x, test_y,
                                       batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # Generate predictions from the model
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(
        input_fn=lambda: eval_input_fn(predict_x,
                                       labels=None,
                                       batch_size=batch_size))

    old_predictions = []
    template = '\nPrediction is "{}" ({:.1f}%), expected "{}"'

    for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(SPECIES[class_id],
                              100 * probability, expec))

        old_predictions.append(SPECIES[class_id])

    # Creating output tf.Variables to specify the output of the saved model.
    feat_specifications = {
        'SepalLength': tf.Variable([], dtype=tf.float64, name="SepalLength"),
        'SepalWidth': tf.Variable([], dtype=tf.float64, name="SepalWidth"),
        'PetalLength': tf.Variable([], dtype=tf.float64, name="PetalLength"),
        'PetalWidth': tf.Variable([], dtype=tf.float64, name="PetalWidth")
    }

    receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feat_specifications)
    temp = tempfile.mkdtemp()
    try:
        # The model is automatically logged when export_saved_model() is called.
        saved_estimator_path = classifier.export_saved_model(temp, receiver_fn).decode("utf-8")
        print(saved_estimator_path)
        # Since the model was automatically logged as an artifact (more specifically
        # a MLflow Model), we don't need to use saved_estimator_path to load back the model.
        # MLflow takes care of it!
        pyfunc_model = pyfunc.load_model(mlflow.get_artifact_uri('model'))

        predict_data = [[5.1, 3.3, 1.7, 0.5], [5.9, 3.0, 4.2, 1.5], [6.9, 3.1, 5.4, 2.1]]
        df = pd.DataFrame(data=predict_data, columns=["SepalLength", "SepalWidth",
                                                      "PetalLength", "PetalWidth"])

        # Predicting on the loaded Python Function and a DataFrame containing the
        # original data we predicted on.
        predict_df = pyfunc_model.predict(df)

        # Checking the PyFunc's predictions are the same as the original model's predictions.
        template = '\nOriginal prediction is "{}", reloaded prediction is "{}"'
        for expec, pred in zip(old_predictions, predict_df['classes']):
            class_id = predict_df['class_ids'][predict_df.loc[predict_df['classes'] == pred].index[0]]
            reloaded_label = SPECIES[class_id]
            print(template.format(expec, reloaded_label))
    finally:
        shutil.rmtree(temp)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_qesfeaw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. 

/opt/conda/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/canned/optimizers.py:102: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  signature = inspect.getargspec(opt.__init__)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp_qesfeaw/model.ckpt.
INFO:tensorflow:loss = 1.1514211, step = 0
INFO:tensorflow:global_step/sec: 533.859
INFO:tensorflow:loss = 0.92138463, step = 100 (0.202 sec)
INFO:tensorflow:global_step/sec: 727.516
INFO:tensorflow:loss = 0.8737793, step = 200 (0.132 sec)
INFO:tensorflow:global_step/sec: 760.024
INFO:tensorflow:loss = 0.82905334, step = 300 (0.131 sec)
INFO:tensorflow:global_step/sec: 758.221
INFO:tensorflow:loss = 0.77164114, step = 400 (0.132 sec)
INFO:tensorflow:global_step/sec: 728.453
INFO:tensorflow:loss = 0.7299293, step = 500 (0.137 sec)
INFO:tensorflow:global_step/sec: 737.38
INFO:tensorflow:loss = 0.73426265, step = 600 (0.136 sec)
INFO:tensorflow:global_step/sec: 746.606
INFO:tensorflow:loss = 0.755196, step = 700 (0.

2020/03/02 07:16:39 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...
2020/03/02 07:16:39 INFO mlflow.tensorflow: Validation succeeded!


/tmp/tmp42h2bivt/1583133399

Original prediction is "Setosa", reloaded prediction is "Setosa"

Original prediction is "Versicolor", reloaded prediction is "Versicolor"

Original prediction is "Versicolor", reloaded prediction is "Versicolor"
